In [1]:
!pip install transformers datasets accelerate bitsandbytes wandb peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 25.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [2]:
!pip install unsloth -q
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.0/175.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [3]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import wandb

user_secrets = UserSecretsClient()
huggingface_token = user_secrets.get_secret("huggingface")
wandb_token = user_secrets.get_secret("wandb")

# login(huggingface_token)
wandb.login(key=wandb_token)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: frqnc (frqnc-upi). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# Load model directly
from unsloth import FastLanguageModel
import torch

max_seq_length = 512
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 42,
    loftq_config = None,
)
print(model.print_trainable_parameters())

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.12.8 patched 16 layers with 16 QKV layers, 16 O layers and 0 MLP layers.


trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750
None


In [5]:
from datasets import Dataset
import json

# Load the dataset
def load_data(file_path):
    articles = []
    summaries = []
    with open(file_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            # First join words in each sentence, then join sentences in each paragraph
            article_text = ' '.join([' '.join(sentence) for paragraph in data['paragraphs'] for sentence in paragraph])
            summary_text = ' '.join([' '.join(sentence) for sentence in data['summary']])

            articles.append(article_text)
            summaries.append(summary_text)
    return articles, summaries

# Paths to dataset JSONL files
train_file = '../input/indosum/train.01.jsonl'
dev_file = '../input/indosum/dev.01.jsonl'
test_file = '../input/indosum/test.01.jsonl'

# Load data
train_articles, train_summaries = load_data(train_file)
dev_articles, dev_summaries = load_data(dev_file)
test_articles, test_summaries = load_data(test_file)

# Create a Hugging Face Dataset
train_dataset = Dataset.from_dict({"document": train_articles, "summary": train_summaries})
val_dataset = Dataset.from_dict({"document": dev_articles, "summary": dev_summaries})
test_dataset = Dataset.from_dict({"document": test_articles, "summary": test_summaries})

In [6]:
data_prompt = """
### SUMMARIZATION TASK ###
- Summarize the new text into 2–3 sentences.
- Focus on the key points.
- If there are examples IGNORE it.
- Generate the summary in Bahasa Indonesia.
- Ignore any news source or site.

Text: {}  
Summary: {}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs = examples["document"]
    outputs = examples["summary"]
    texts = []
    labels = []

    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
        labels.append(output)  # Keep the raw output for tokenization below

    # Tokenize inputs and labels
    tokenized_inputs = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )
    tokenized_labels = tokenizer(
        labels,
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )

    # Replace padding token id in labels with -100 to ignore in loss computation
    labels_with_ignore = tokenized_labels["input_ids"].clone()
    labels_with_ignore[labels_with_ignore == tokenizer.pad_token_id] = -100

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": labels_with_ignore
    }

In [7]:
train_dataset = train_dataset.map(formatting_prompt, batched=True, remove_columns=["document", "summary"])
val_dataset = val_dataset.map(formatting_prompt, batched=True, remove_columns=["document", "summary"])

Map:   0%|          | 0/14262 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Update training arguments for distributed training
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    eval_strategy="epoch",    # Evaluate at the end of each epoch
    learning_rate=3e-4,             # Learning rate
    gradient_accumulation_steps=2,  # Accumulate gradients over more steps
    per_device_train_batch_size=8,  # Reduce batch size to fit memory
    per_device_eval_batch_size=8,   # Reduce eval batch size
    num_train_epochs=5,             # Number of epochs
    save_strategy="epoch",          # Save model at the end of each epoch
    logging_dir="./logs",           # Logging directory
    logging_steps=100,              # Log every 100 steps
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    optim="adamw_8bit",      # Optimizer
    report_to="wandb",              # Enable logging to WandB
    remove_unused_columns=False
)

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    max_length=max_seq_length,
    label_pad_token_id=-100  # Ensure labels are padded with -100
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = data_collator,
    dataset_num_proc = 2,
    packing = False,
    args = training_args
)

In [10]:
torch.cuda.empty_cache()
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 14,262 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 4,455
 "-____-"     Number of trainable parameters = 3,407,872
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241222_091359-h6magebj
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./results
wandb: ⭐️ View project at https://wandb.ai/frqnc-upi/huggingface
wandb: 🚀 View run at https://wandb.ai/frqnc-upi/huggingface/runs/h6magebj


Epoch,Training Loss,Validation Loss
0,7.063700,7.060445
2,6.976400,7.016500
4,6.856000,7.031026


TrainOutput(global_step=4455, training_loss=6.985517378099572, metrics={'train_runtime': 18026.8344, 'train_samples_per_second': 3.956, 'train_steps_per_second': 0.247, 'total_flos': 2.1381413544237466e+17, 'train_loss': 6.985517378099572, 'epoch': 4.997195737521032})

In [11]:
tokenizer.save_pretrained("./results/model")
model.save_pretrained("./results/model", "lora_model")